In [1]:
import numpy as np

![Метод оптимального исключения](elimination.jpg)

In [2]:
def optimal_elimination(a: np.ndarray, b: np.ndarray, show_annotation: bool = False) -> np.ndarray:
    
    if a.shape[0] != a.shape[1]:
        raise ValueError("Matrix a must be square")
    
    if a.shape[0] != b.shape[0]:
        raise ValueError("Dimensions of a and b must match")
    
    det_a = np.linalg.det(a)
    if abs(det_a) < 1e-12:
        raise ValueError("Determinant is near zero — no unique solution")
    
    n = len(b)
    
    ab = np.zeros((n, n + 1), dtype=float)
    ab[:, :n] = a.astype(float)
    ab[:, n] = b.astype(float)

    cols = np.arange(n, dtype=int)

    if show_annotation:
        print("System:\n", ab, "\ncols:", cols, "\n")

    for k in range(n):
        p = k + int(np.argmax(np.abs(ab[k, k:n])))

        if p != k:
            ab[:, [k, p]] = ab[:, [p, k]]
            cols[[k, p]] = cols[[p, k]]
            if show_annotation:
                print(f"Step {k+1}: swap columns {k} and {p}")

        ab_next = ab.copy()

        if k == 0:
            denom = ab[0, 0]
        else:
            denom = ab[k, k] - sum(ab[r, k] * ab[k, r] for r in range(0, k))

        # k-я строка для столбцов p = k..n 
        # (2)
        for p in range(0, n + 1):
            numer = ab[k, p]
            if k > 0:
                numer -= sum(ab[r, p] * ab[k, r] for r in range(0, k))
            ab_next[k, p] = numer / denom

        # i = 0..k-1, p = k+1..n 
        # (3)
        for i in range(0, k):
            for p in range(k + 1, n + 1):
                ab_next[i, p] = ab[i, p] - ab_next[k, p] * ab[i, k]
            ab_next[i, k] = 0

        ab_next[k, k] = 1

        ab = ab_next

        if show_annotation:
            print(f"After step {k+1}:")
            print(ab)
            print("cols:", cols, "\n")

    x_perm = ab[:, n].copy()

    ans = np.zeros(n, dtype=float)
    for j in range(n):
        ans[cols[j]] = x_perm[j]

    return ans

In [3]:
a = np.array([[2, 3, 2], [2, 3, 4], [3, 4, 5]])
b = np.array([7, 9, 12])
optimal_elimination(a, b, show_annotation=True)

System:
 [[ 2.  3.  2.  7.]
 [ 2.  3.  4.  9.]
 [ 3.  4.  5. 12.]] 
cols: [0 1 2] 

Step 1: swap columns 0 and 1
After step 1:
[[ 1.          0.66666667  0.66666667  2.33333333]
 [ 3.          2.          4.          9.        ]
 [ 4.          3.          5.         12.        ]]
cols: [1 0 2] 

Step 2: swap columns 1 and 2
After step 2:
[[ 1.          0.          0.66666667  1.66666667]
 [ 0.          1.          0.          1.        ]
 [ 4.          5.          3.         12.        ]]
cols: [1 2 0] 

After step 3:
[[1. 0. 0. 1.]
 [0. 1. 0. 1.]
 [0. 0. 1. 1.]]
cols: [1 2 0] 



array([1., 1., 1.])

In [8]:
systems = [
    (np.array([[2, 3], [1, -1]], dtype=float), np.array([8, 0], dtype=float)),
    
    (np.array([[2, 3, 2],
               [2, 3, 4],
               [3, 4, 5]], dtype=float), np.array([7, 9, 12], dtype=float)),
    
    (np.array([[4, 2, 0, 1],
               [3, 1, -1, 2],
               [2, -1, 3, 1],
               [1, 0, 2, 1]], dtype=float), np.array([10, 8, 7, 3], dtype=float)),
    
    (np.array([[2, 1, 3, 2, 4],
               [1, 2, 0, 1, 3],
               [3, 0, 2, 1, 1],
               [2, 1, 1, 3, 2],
               [1, 3, 2, 0, 1]], dtype=float), np.array([10, 6, 7, 9, 5], dtype=float)),
    
    (np.array([[1, 2, 1, 3, 2, 1],
               [2, 1, 3, 1, 2, 1],
               [3, 2, 1, 2, 1, 3],
               [1, 3, 2, 1, 3, 2],
               [2, 1, 2, 3, 1, 2],
               [1, 2, 3, 1, 2, 1]], dtype=float), np.array([10, 11, 12, 13, 14, 15], dtype=float)),
]


np.random.seed(42)
systems.append((np.random.rand(10, 10) * 10, np.random.rand(10) * 10))
systems.append((np.random.rand(20, 20) * 10, np.random.rand(20) * 10))

for idx, (A, b) in enumerate(systems, start=1):
    print(f"\n{A.shape[0]}x{A.shape[1]}")
    try:
        result = optimal_elimination(A, b)
        expected = np.linalg.solve(A, b)
        print("Максимальная абсолютная погрешность:", np.max(np.abs(result - expected)))
        
    except ValueError as e:
        print(e)
        

result = optimal_elimination(systems[-2][0], systems[-2][1], show_annotation=True)


2x2
Максимальная абсолютная погрешность: 4.440892098500626e-16

3x3
Максимальная абсолютная погрешность: 5.773159728050814e-15

4x4
Максимальная абсолютная погрешность: 0.0

5x5
Максимальная абсолютная погрешность: 8.881784197001252e-16

6x6
Максимальная абсолютная погрешность: 1.4432899320127035e-15

10x10
Максимальная абсолютная погрешность: 1.509903313490213e-14

20x20
Максимальная абсолютная погрешность: 8.43769498715119e-14
System:
 [[3.74540119 9.50714306 7.31993942 5.98658484 1.5601864  1.5599452
  0.58083612 8.66176146 6.01115012 7.08072578 0.31429186]
 [0.20584494 9.69909852 8.32442641 2.12339111 1.81824967 1.8340451
  3.04242243 5.24756432 4.31945019 2.9122914  6.36410411]
 [6.11852895 1.39493861 2.92144649 3.66361843 4.56069984 7.85175961
  1.99673782 5.14234438 5.92414569 0.46450413 3.14355981]
 [6.07544852 1.70524124 0.65051593 9.48885537 9.65632033 8.08397348
  3.04613769 0.97672114 6.84233027 4.40152494 5.08570691]
 [1.22038235 4.9517691  0.34388521 9.09320402 2.5877998

In [6]:
import time
def test_large_system(n):
    print(f"\n{n}x{n}")
    np.random.seed(42)
    A = np.random.rand(n, n) * 10
    b = np.random.rand(n) * 10

    start = time.time()
    result = optimal_elimination(A, b)
    elapsed_gauss = time.time() - start
    
    start = time.time()
    expected = np.linalg.solve(A, b)
    elapsed_numpy = time.time() - start

    max_error = np.max(np.abs(result - expected))
    print(f"Максимальная абсолютная погрешность: {max_error:.2e}")
    print(f"Время Гаусс = {elapsed_gauss:.4f}s, NumPy = {elapsed_numpy:.4f}s")

test_large_system(50)
test_large_system(100)
test_large_system(300)
test_large_system(500)
test_large_system(800)


50x50
Максимальная абсолютная погрешность: 3.58e-13
Время Гаусс = 0.1951s, NumPy = 0.0005s

100x100
Максимальная абсолютная погрешность: 2.85e-12
Время Гаусс = 0.6490s, NumPy = 0.0000s

300x300
Максимальная абсолютная погрешность: 7.37e-12
Время Гаусс = 13.1800s, NumPy = 0.0010s

500x500
Максимальная абсолютная погрешность: 2.25e-11
Время Гаусс = 56.0187s, NumPy = 0.0168s

800x800


KeyboardInterrupt: 